## Calcul du Hurst par régression

In [28]:
import numpy as np
from scipy import optimize
import pandas as pd
from scipy.stats import norm
import warnings 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd


In [3]:
file = "Data issuers.xlsx"
market_cap = pd.read_excel(file, sheet_name="Mod Market Cap")
market_cap = market_cap.set_index("Dates").loc['2019-10-28':'2020-10-13']
debt = pd.read_excel(file, sheet_name="Gross Debt", nrows=1)

In [26]:
# Définition de la fonction genhurst avec les arguments S (série temporelle) et q (ordre)
def genhurst(S, q):

    # Obtient la longueur de la série temporelle
    L = len(S)

    # Avertissement si la série temporelle est très courte (moins de 100 points)
    if L < 100:
        warnings.warn('Data series very short!')

    # Initialisation d'un tableau H pour stocker les résultats du coefficient de Hurst
    H = np.zeros((len(range(5, 20)), 1))  # Crée un tableau de zéros de forme (15, 1)
    k = 0  # Initialise un compteur

    # Boucle sur différentes fenêtres de temps (Tmax)
    for Tmax in range(5, 20):

        # Génère une séquence de nombres de 1 à Tmax
        x = np.arange(1, Tmax + 1, 1)

        # Initialise un tableau mcord pour stocker les résultats locaux du coefficient de Hurst
        mcord = np.zeros((Tmax, 1))  # Crée un tableau de zéros de forme (Tmax, 1)

        # Boucle à travers les décalages temporels (tt) dans la fenêtre actuelle
        for tt in range(1, Tmax + 1):

            # Calcule les différences et les valeurs retardées
            dV = S[np.arange(tt, L, tt)] - S[np.arange(tt, L, tt) - tt]
            VV = S[np.arange(tt, L + tt, tt) - tt]
            N = len(dV) + 1
            X = np.arange(1, N + 1, dtype=np.float64)
            Y = VV

            # Calcul des coefficients pour ajuster une droite
            mx = np.sum(X) / N
            SSxx = np.sum(X**2) - N * mx**2
            my = np.sum(Y) / N
            SSxy = np.sum(np.multiply(X, Y)) - N * mx * my
            cc1 = SSxy / SSxx
            cc2 = my - cc1 * mx
            ddVd = dV - cc1
            VVVd = VV - np.multiply(cc1, np.arange(1, N + 1, dtype=np.float64)) - cc2

            # Calcul du coefficient de Hurst local
            mcord[tt - 1] = np.mean(np.abs(ddVd)**q) / np.mean(np.abs(VVVd)**q)

        # Régression linéaire sur le logarithme des échelles
        mx = np.mean(np.log10(x))
        SSxx = np.sum(np.log10(x)**2) - Tmax * mx**2
        my = np.mean(np.log10(mcord))
        SSxy = np.sum(np.multiply(np.log10(x), np.transpose(np.log10(mcord)))) - Tmax * mx * my

        # Stocke le résultat du coefficient de Hurst dans le tableau H
        H[k] = SSxy / SSxx
        k = k + 1

    # Calcule la moyenne des coefficients de Hurst sur toutes les fenêtres de temps et divise par q
    mH = np.mean(H) / q

    return mH  # Retourne le coefficient de Hurst moyen divisé par l'ordre q


def affichage_coef_hurst(col_name, market_cap_data):
    # Extrait les données de la colonne spécifiée
    market_cap_array = market_cap_data[col_name].to_numpy()

    # Initialise une liste pour stocker les résultats du coefficient de Hurst pour chaque mois
    hurst_coefficients = []

    # Loop à travers chaque mois
    for i in range(0, len(market_cap_array), 1):
        # Extrait la market cap pour le mois actuel
        market_cap_month = market_cap_array[:i + 1]

        # Calcule le coefficient de Hurst en utilisant la fonction genhurst
        hurst_coefficient = genhurst(market_cap_month, q=2)  # Vous pouvez ajuster la valeur de q si nécessaire

        # Stocke le résultat dans la liste
        hurst_coefficients.append(hurst_coefficient)

    # Crée un dataframe pour Plotly Express
    plot_df = pd.DataFrame({
                            'Hurst Coefficient': hurst_coefficients,
                            col_name: market_cap_array[:len(hurst_coefficients)]})

    # Crée un subplot avec deux axes Y
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Ajoute la première ligne avec l'axe Y principal
    fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df[col_name], mode='lines', name=col_name), secondary_y=False)
    
    # Ajoute la deuxième ligne avec l'axe Y secondaire
    fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['Hurst Coefficient'], mode='lines', name='Hurst Coefficient'), secondary_y=True)

    # Met à jour les étiquettes et le titre
    fig.update_layout(title_text=f"{col_name} and Hurst Coefficient over Time")
    fig.update_xaxes(title_text='Time')
    fig.update_yaxes(title_text=col_name, secondary_y=False)
    fig.update_yaxes(title_text='Hurst Coefficient', secondary_y=True)

    # Affiche le graphique
    fig.show()


In [27]:
# Créer un nouveau dataframe pour stocker les résultats
result_list = []

# Appel de la fonction avec le nom de la colonne et stocke le résultat dans la liste
result_list.append(affichage_coef_hurst('FGR FP Equity', market_cap))

# Affiche la liste des résultats
print(result_list)


C:\Users\louis\AppData\Local\Temp\ipykernel_11476\1022306552.py:12: UserWarning:

Data series very short!

C:\Users\louis\AppData\Local\Temp\ipykernel_11476\1022306552.py:42: RuntimeWarning:

invalid value encountered in scalar divide

c:\Users\louis\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning:

Mean of empty slice.

c:\Users\louis\anaconda3\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning:

invalid value encountered in scalar divide

C:\Users\louis\AppData\Local\Temp\ipykernel_11476\1022306552.py:48: RuntimeWarning:

invalid value encountered in scalar divide



[None]


## Calcul du Hurst Par régression 

In [46]:
import numpy as np
from numpy import sqrt, std, subtract, polyfit, log

def hurst(ts):
    """Returns the Hurst Exponent of the time series vector ts"""
    if len(ts)< 100 : 
        return np.nan
    # Create the range of lag values
    lags = range(2, 100)

    # Calculate the array of the variances of the lagged differences
    tau = [sqrt(std(subtract(ts[lag:], ts[:-lag]))) for lag in lags]

    # Use a linear fit to estimate the Hurst Exponent
    poly = polyfit(log(lags), log(tau), 1)

    # Return the Hurst exponent from the polyfit output
    return poly[0] * 2.0


def affichage_coef_hurst_reg(col_name, market_cap_data):
    # Extrait les données de la colonne spécifiée
    market_cap_array = market_cap_data[col_name].to_numpy()

    # Initialise une liste pour stocker les résultats du coefficient de Hurst pour chaque mois
    hurst_coefficients = []

    # Loop à travers chaque mois
    for i in range(0, len(market_cap_array), 1):
        # Extrait la market cap pour le mois actuel
        market_cap_month = market_cap_array[:i + 1]

        # Calcule le coefficient de Hurst en utilisant la fonction genhurst
        hurst_coefficient = hurst(market_cap_month)  # Vous pouvez ajuster la valeur de q si nécessaire

        # Stocke le résultat dans la liste
        hurst_coefficients.append(hurst_coefficient)

    # Crée un dataframe pour Plotly Express
    plot_df = pd.DataFrame({
                            'Hurst Coefficient': hurst_coefficients,
                            col_name: market_cap_array[:len(hurst_coefficients)]})

    # Crée un subplot avec deux axes Y
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Ajoute la première ligne avec l'axe Y principal
    fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df[col_name], mode='lines', name=col_name), secondary_y=False)
    
    # Ajoute la deuxième ligne avec l'axe Y secondaire
    fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['Hurst Coefficient'], mode='lines', name='Hurst Coefficient'), secondary_y=True)

    # Met à jour les étiquettes et le titre
    fig.update_layout(title_text=f"{col_name} and Hurst Coefficient over Time")
    fig.update_xaxes(title_text='Time')
    fig.update_yaxes(title_text=col_name, secondary_y=False)
    fig.update_yaxes(title_text='Hurst Coefficient', secondary_y=True)

    # Affiche le graphique
    fig.show()

In [47]:
# Créer un nouveau dataframe pour stocker les résultats
result_list = []

# Appel de la fonction avec le nom de la colonne et stocke le résultat dans la liste
result_list.append(affichage_coef_hurst_reg('FGR FP Equity', market_cap))

# Affiche la liste des résultats
print(result_list)


C:\Users\louis\AppData\Local\Temp\ipykernel_11476\3013531256.py:15: RuntimeWarning:

divide by zero encountered in log



[None]
